# CO oxidation in a CSTR

## Load input files

In [1]:
from pycatkin.functions.load_input import read_from_input_file

sim_system_Au = read_from_input_file(input_path='input_AuPd.json')
sim_system_Pd = read_from_input_file(input_path='input_Pd111.json')


Loading input file: input_AuPd.json.
Reading states:
* s
* sCO
* sO
* CO
* O2
* CO2
Reading scaling relation states:
* SRTS
Reading system:
* Pressure: 100000 Pa
* Temperature: 423 K
Reading reactions:
* CO_ads
* O2_ads
* CO_ox
Reading reactor:
* CSTReactor
Done.
Loading input file: input_Pd111.json.
Reading states:
* s
* sCO
* sO
* CO
* O2
* CO2
Reading scaling relation states:
* SRTS
Reading system:
* Pressure: 100000 Pa
* Temperature: 423 K
Reading reactions:
* CO_ads
* O2_ads
* CO_ox
Reading reactor:
* CSTReactor
Done.


In [ ]:
sim_system_Au.

## View the states using ASE

In [2]:
from pycatkin.functions.presets import draw_states
import os

if not os.path.isdir('figures'):
    os.mkdir('figures')

draw_states(sim_system=sim_system_Au,
            fig_path='figures/AuPd/')  # rotation='-90x'


Directory does not exist. Will try creating it...


AttributeError: 'System' object has no attribute 'snames'

## Save the states in proteindatabank (.pdb) format

In [ ]:
import os

if not os.path.isdir('figures'):
    os.mkdir('figures')

for s in sim_system_Pd.snames:
    if sim_system_Pd.states[s].state_type != 'TS':
        sim_system_Pd.states[s].save_pdb(path='figures/Pd111/')


## Run simulations for a range of temperatures

In [ ]:
from pycatkin.functions.presets import run_temperatures
import os
import numpy as np

if not os.path.isdir('outputs'):
    os.mkdir('outputs')

temperatures = np.linspace(start=423, stop=623, num=20, endpoint=True)

for sysname, sim_system in [['AuPd', sim_system_Au], ['Pd111', sim_system_Pd]]:
    run_temperatures(sim_system=sim_system,
                     temperatures=temperatures,
                     steady_state_solve=True,
                     plot_results=False,
                     save_results=True,
                     fig_path='figures/%s/' % sysname,
                     csv_path='outputs/%s/' % sysname)


## Compute and plot conversion across the reactor

In [ ]:
from pycatkin.functions.presets import plot_data_simple
import os
import pandas as pd

fig, ax = None, None
if not os.path.isdir('figures'):
    os.mkdir('figures')

for sysname, sim_system in [['AuPd', sim_system_Au], ['Pd111', sim_system_Pd]]:

    df = pd.read_csv(filepath_or_buffer='outputs/%s/pressures_vs_temperature.csv' % sysname)

    pCOin = sim_system_Pd.params['inflow_state']['CO']
    pCOout = df['pCO (bar)'].values
    xCO = 100.0 * (1.0 - pCOout / pCOin)

    fig, ax = plot_data_simple(fig=fig,
                               ax=ax,
                               xdata=temperatures,
                               ydata=xCO,
                               xlabel='Temperature (K)',
                               ylabel='Conversion (%)',
                               label=sysname,
                               addlegend=True,
                               color='teal' if sysname == 'Pd111' else 'salmon',
                               fig_path='figures/',
                               fig_name='conversion')
